# Harley Wood School for Astronomy 2019 

<img src="https://research.smp.uq.edu.au/asa2019/static/asa19/img/HWSA2019-logo.png" width=300>

## Part IV - Timing and profiling code snippets to help write faster code

In this session, you will learn how to measure the runtime 


## Table of Contents

1. [When and What to Optimise](#When-and-what-to-optimise)
2. [Timing Code Snippets](#Timing-code-snippets)
3. [Profiling Code](#Profiling-code-line-by-line)
4. [Parallelising Code](#Parallelising-code)
5. [Advanced Track](#Advanced-Track)


### Required libraries

This notebook uses several Python packages that come standard with the [Anaconda Python distribution](http://continuum.io/downloads). The primary libraries that we'll be using are:

* **astropy**
* **pandas**
* **numpy**
* **pandas**
* **line_profiler**
* **memory_profiler**
* **schwimmbad** [optional]

If you have created a new environment with `conda env create -f hwsa-environment.yml`, then you are all set. If not, To make sure you have all of the packages you need, install them with `conda`:

    conda install [package name]
    
`conda` may ask you to update some of the packages if you don't have the most recent version. Allow it to do so.

Alternatively, if you can install the packages with [pip](https://pip.pypa.io/en/stable/installing/) (a Python package manager):

    pip install [package name]
    
Be sure to restart your kernel if you had to install new packages.

# When and What to Optimise


**Code is read far more frequently than written**

Optimisation mantras:

1. [Obligatory XKCD](https://xkcd.com/1205/)

2. Do not optimise unless you have to (your time is *the most precious*)

3. Optimise for readability (imagine very cranky future developers, and reduce their cognitive load)

4. Variety of optimisations 
    - Readability
    - Usability
    - Performance   
    - Memory
    - Lines of code (code-golf)
    
5. Remember, every line of code is a potential source of bugs, and future maintenance head-aches. Choose wisely!    


# Timing Code Snippets

`time` magic command measures the time taken by a code snippet by running the snippet once. What are the challenges to timing a piece of code you say?

- Ensure that the timing is representative of the average case
- Get some sense of the scatter in the runtimes

`timeit` magic command tries to answer both those potential issues. `timeit` will repeat a chunk of code some *N* times till a stable runtime measurement is reached. Lots of customisable options -- see the manual [here](https://docs.python.org/3/library/timeit.html)

# Magic commands tips

Use `%%` at the top of the code snippet (if not a function), or use `%` to invoke when timing a function.

In [2]:
%%timeit
# read data 
from astropy.io.votable import parse_single_table
table = parse_single_table("async_20190630210155.vot")
#t = table.to_table(use_names_over_ids=True)

KeyboardInterrupt: 

# That takes a LOOOONG time

Can we reduce the total read time? Perhaps we can only read-in the specific columns that we *need*

In [1]:
%%timeit
#read the subset data 
from astropy.io.votable import parse_single_table
table = parse_single_table("async_subset.vot")
t = table.to_table(use_names_over_ids=True)

2.79 s ± 196 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
## If you downloaded the pickle file, you can restore via this cell

#import pandas as pd
#df = pd.read_pickle("async_20190630210155.pkl")

In [2]:
#read data -> but this try/except will protect you from re-running a very SLOOW process (un-necessarily)
try:
    table
except NameError:
    from astropy.io.votable import parse_single_table
    columns = ['phot_g_mean_mag', 'parallax']
    table = parse_single_table("async_subset.vot", columns=columns)
    print("Done reading table")

t = table.to_table(use_names_over_ids=True)

Done reading table


# Converting the data

Now that you have managed to read-in the data (either the full or subset one), let's convert this to a more convenient data-frame

In [3]:
%%time
df = t.to_pandas()

#check the data frame
df.head()

CPU times: user 264 ms, sys: 110 ms, total: 375 ms
Wall time: 1.73 s


/anaconda3/lib/python3.7/site-packages/IPython/core/magics/execution.py:1271: TableReplaceWarning: converted column 'solution_id' from integer to float
  exec(code, glob, local_ns)
/anaconda3/lib/python3.7/site-packages/IPython/core/magics/execution.py:1271: TableReplaceWarning: converted column 'source_id' from integer to float
  exec(code, glob, local_ns)
/anaconda3/lib/python3.7/site-packages/IPython/core/magics/execution.py:1271: TableReplaceWarning: converted column 'random_index' from integer to float
  exec(code, glob, local_ns)
/anaconda3/lib/python3.7/site-packages/IPython/core/magics/execution.py:1271: TableReplaceWarning: converted column 'astrometric_n_obs_al' from integer to float
  exec(code, glob, local_ns)
/anaconda3/lib/python3.7/site-packages/IPython/core/magics/execution.py:1271: TableReplaceWarning: converted column 'astrometric_n_obs_ac' from integer to float
  exec(code, glob, local_ns)
/anaconda3/lib/python3.7/site-packages/IPython/core/magics/execution.py:1271: 

In [4]:
%%timeit
#convert to pandas df and calculate absolute mag
import numpy as np
from math import log10

df['mg'] = 0
df['dist'] = 0

for c, v in enumerate(df['phot_g_mean_mag']):
    
    p =df.loc[c,'parallax']
    if p>0:
        df.loc[c,'mg'] = v + 5 * log10(p) - 10
        df.loc[c,'dist'] = 1000/p
    else:
        df.loc[c,'mg'] = np.nan
        df.loc[c,'dist'] = np.nan

5.71 s ± 6.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# For loops are slow 

In python, `for` loops are slow. So you should try to avoid them whenever possible. In the following example, we will try to create an array of integers, where each element is the square of the index. The goal would be to create the fastest possible implementation. 

In [5]:
def basic_list_creation(N):    
    squares = []
    for i in range(N):
        squares.append(i**2)
        
    return squares

In [7]:
def list_comprehension(N):
    squares = [i**2 for i in range(N)] # They call this list comperehension.
    return squares

In [8]:
N = 10000000

In [9]:
%timeit basic_list_creation(N)

3.31 s ± 3.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit list_comprehension(N)

2.95 s ± 5.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Allocating and re-allocating memory is slow

If you know the size of the array, then pre-allocate the array/list, and then fill in at the appropriate index. 

In [11]:
def prealloc_list(N):
    # Creating a list of N zeroes.
    squares = [0]*N
    for i in range(N):
        squares[i] = i*i

In [12]:
%timeit prealloc_list(N)

879 ms ± 790 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Numpy is faster

Let's try to write this "square" function in `numpy`

In [15]:
def numpy_squares(N):
    import numpy as np
    squares = np.empty(N)
    for i in range(N):
        squares[i] = i*i

In [16]:
%timeit numpy_squares(N)

1.33 s ± 34.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
# For advanced users -> implement this function. Extra points if you can make `numpy_only` more than 20x faster
import numpy as np
def numpy_only(N):
    x = np.arange(N)
    return x*x

In [22]:
%timeit numpy_only(N)

30.4 ms ± 2.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Back to processing GAIA data

In [23]:
def second_attempt_at_abs_mag_and_dist(df):
    import pandas as pd
    import numpy as np
    import math

    df['mg2'] = 0
    df['dist2'] = 0

    for c, v in enumerate(df['phot_g_mean_mag']):

        p =df.loc[c,'parallax']
        if p>0:
            df.loc[c,'mg2'] = v + 5 * math.log10(p) - 10
            df.loc[c,'dist2'] = 1000/p
        else:
            df.loc[c,'mg2'] = np.nan
            df.loc[c,'dist2'] = np.nan

In [24]:
%timeit second_attempt_at_abs_mag_and_dist(df)

5.91 s ± 244 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## For loops are sloooow in python

But we do need to loop - how do we that? List comprehensions to the rescue!

In [25]:
#convert to pandas df and calculate absolute mag
def third_attempt_at_abs_mag_and_dist(df):
    import numpy as np
    import math

    apparent_mags = df['phot_g_mean_mag']
    parallax = df['parallax']
    abs_mags = [mag + 5*math.log10(dist) - 10 if dist > 0 else np.nan for mag, dist in zip(apparent_mags, parallax)]
    dists = [1000.0/d if d > 0 else np.nan for d in parallax ]
    
    df['mg3'] = abs_mags
    df['dist3'] = dists

In [26]:
%timeit third_attempt_at_abs_mag_and_dist(df)

4.29 ms ± 81.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
#convert to pandas df and calculate absolute mag
def fourth_attempt_at_abs_mag_and_dist(df):
    import numpy as np

    apparent_mags = df['phot_g_mean_mag']
    parallax = df['parallax']
    abs_mags = [mag + 5*np.log10(dist) - 10 if dist > 0 else np.nan for mag, dist in zip(apparent_mags, parallax)]
    dists = [1000.0/d if d > 0 else np.nan for d in parallax ]
    
    df['mg4'] = abs_mags
    df['dist4'] = dists

%timeit fourth_attempt_at_abs_mag_and_dist(df)

15 ms ± 737 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
#convert to pandas df and calculate absolute mag
def fifth_attempt_at_abs_mag_and_dist(df):
    import pandas as pd
    import numpy as np
    import math

    apparent_mags = df['phot_g_mean_mag']
    parallax = df['parallax']
    abs_mags = apparent_mags + 5.0*np.log10(parallax) - 10
    dist = 1000.0/parallax

    bad_inds = (~np.isfinite(parallax) | (parallax <= 0))
    abs_mags[bad_inds] = np.nan
    dist[bad_inds] = np.nan
  
    df['mg5'] = abs_mags
    df['dist5'] = dist

%timeit fifth_attempt_at_abs_mag_and_dist(df)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log10
  if __name__ == '__main__':


4.18 ms ± 25.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
#convert to pandas df and calculate absolute mag
def sixth_attempt_at_abs_mag_and_dist(df):
    import numpy as np

    apparent_mags = df['phot_g_mean_mag']
    parallax = df['parallax']
    
    abs_mags = np.full_like(apparent_mags, np.nan)
    dist = np.full_like(parallax, np.nan)

    good_inds = (np.isfinite(parallax) & (parallax > 0))
    abs_mags[good_inds] = apparent_mags[good_inds] + 5.0*np.log10(parallax[good_inds]) - 10
    dist[good_inds] = 1000.0/parallax[good_inds]

    df['mg6'] = abs_mags
    df['dist6'] = dist

%timeit sixth_attempt_at_abs_mag_and_dist(df)

2.01 ms ± 6.49 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
def add_abs_mag_and_distance(df):
    import numpy as np
    df['optim_abs_mag'] = df['phot_g_mean_mag'] + 5*np.log10(df['parallax']) - 10
    df['optim_dist'] = 1000.0/df['parallax']
       
%timeit  add_abs_mag_and_distance(df)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until


839 µs ± 13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Profiling Code (line by line)

So far we have written code and timed entire chunks of code. What if you wanted to know how much time each line of your code takes?

There is a magic command for that -- `line_profiler`. But now you do have to load this magic command

In [1]:
%load_ext line_profiler

In [34]:
def second_attempt_at_abs_mag_and_dist(df):
    import pandas as pd
    import numpy as np
    import math

    df['mg2'] = 0
    df['dist2'] = 0

    for c, v in enumerate(df['phot_g_mean_mag']):

        p =df.loc[c,'parallax']
        if p>0:
            df.loc[c,'mg2'] = v + 5 * math.log10(p) - 10
            df.loc[c,'dist2'] = 1000/p
        else:
            df.loc[c,'mg2'] = np.nan
            df.loc[c,'dist2'] = np.nan

In [35]:
%lprun -f second_attempt_at_abs_mag_and_dist second_attempt_at_abs_mag_and_dist(df)

In [36]:
%lprun -f third_attempt_at_abs_mag_and_dist third_attempt_at_abs_mag_and_dist(df)

In [37]:
%lprun -f add_abs_mag_and_distance add_abs_mag_and_distance(df)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until


# An application of line-profiling

## Finding Unique Values

Say you might find the number of unique values in an HUGE array, where potentially many elements are repeated.

In [2]:
def find_unique_values(arr):
    unique_vals = []
    
    for x in arr:
        if x not in unique_vals:
            unique_vals.append(x)
    
    return unique_vals    

In [5]:
import numpy as np
N = 1000
nrepeats = 300
dups = np.repeat(np.arange(N), nrepeats)

%timeit find_unique_values(dups)

2.74 s ± 23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
dups.shape

(300000,)

In [11]:
def find_mine_unique_values(arr):
    unique_vals = np.empty((arr.shape))
    unique_vals[:] = np.nan
    unique_vals[i] = [arr[i] for x, i in (arr, range(len(arr)))]

In [13]:
# find_mine_unique_values(dups)

# Exercise

Use line-profiler oon this above code and come up with faster solutions (that still use `for` loops). Bonus points for solutions without `for` loops. 

# Idiomatic python 

`if x in large_array` is `O(len(large_array))` if `large_array` is a list.

# How about profiling for memory usage

What is the amount of memory required per line of code.

In [15]:
%load_ext memory_profiler

In [20]:
%%file mprun_demo.py
def read_votable(table=False):
    if not table:
        from astropy.io.votable import parse_single_table
        columns = ['phot_g_mean_mag', 'parallax']
        table = parse_single_table("async_subset.vot", columns=columns)
        print("Done reading table")
    return table.to_table(use_names_over_ids=True)

Overwriting mprun_demo.py


In [21]:
from mprun_demo import read_votable
%mprun -f read_votable read_votable()

UnboundLocalError: local variable 'table' referenced before assignment

# Parallelising code

Python, **by design**, will only run in serial. There are a couple of ways you can run code in parallel. Most commonly, you will use the `multiprocessing` module. 

*Note* `multiprocessing` has a large overhead for creating these new "processes". If you are trying to use `multiprocessing` for relatively fast code snippets, then chances are that the "parallel" version will be slower.

In [22]:
import multiprocessing
print("Number of cpus = {}".format(multiprocessing.cpu_count()))

Number of cpus = 4


In [23]:
try:
    xrange
except NameError:
    xrange = range

def squareit(x):
    return x*x

In [24]:
import numpy as np
N=100000

%timeit squareit(np.arange(N))

73.6 µs ± 65.6 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
%%timeit
from multiprocessing import Pool
nprocs = multiprocessing.cpu_count()
with Pool(processes=nprocs) as pool:
    _  = pool.map(squareit, range(N))

123 ms ± 589 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit 

from schwimmbad import MultiPool

with MultiPool(nprocs) as pool:
    _ = list(pool.map(squareit, range(N)))

In [ ]:
import multiprocessing

In [ ]:
%%timeit 

from schwimmbad import MultiPool

with MultiPool(nprocs) as pool:
    _ = list(pool.map(squareit, range(N)))

# MPI parallelism

As long as you can write a function that operates on a certain chunk of the data, then both `multiprocessing` and `MPI` based parallelism are feasible. `mpi4py` is a convenient package that lets you use a supercomputer cluster, and in theory, your code can run significantly faster (limited by the number of cores on your supercomputer). 

I would recommend using the `schwimmbad` package to transparently use both `MPI` and `multiprocessing` based parallelism. 

# Advanced Track 


## Challenge #1: Difficulty Rating - *Medium*

Perform the entire operation of calculating `absolute-mag` and `distance` in parallel. First, create the parallel interface with `multiprocessing`, and then create an MPI parallel implementation with `mpi4py`.

Hint: Use the package `schwimmbad` for a customisable solution. 


## Challenge #2: Difficulty Rating - *Easy*

There is an open issue on the astropy repo [issue no 8946](https://github.com/astropy/astropy/issues/8946). Figure out what causes the high-memory usage. 

Hint: You will need `mem_profiler` for the first challenge


## Challenge #3: Difficulty Rating - *Medium*

Open a pull-request to fix [issue no 8946](https://github.com/astropy/astropy/issues/8946)


## Challenge #4: Difficulty Rating - *Ninja*

Open a pull-request to fix [issue no 8946](https://github.com/astropy/astropy/issues/8946) *AND* to read-in only some *Nrows* of data (i.e., add a new keyword). Any addition must maintain backwards compatibility. 

**Disclaimer** (MS) I do not know how to do this. If you are (interested in) attempting this, please let me know - I can put you in touch with astropy developers.